In [35]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import base64
import re
from google_auth_oauthlib.flow import InstalledAppFlow
from email.message import EmailMessage

In [36]:
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly','https://www.googleapis.com/auth/gmail.modify']

creds = None
# The file token.json stores the user's access and refresh tokens,and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
	creds = Credentials.from_authorized_user_file('token.json',SCOPES)
# If there are no (valid) credentials available,let the user log in.
if not creds or not creds.valid:
	if creds and creds.expired and creds.refresh_token:
		creds.refresh(Request())
	else:
		flow = InstalledAppFlow.from_client_secrets_file(
			'credentials.json',SCOPES)
		creds = flow.run_local_server(port=0)
	# Save the credentials for the next run
	with open('token.json','w') as token:
		token.write(creds.to_json())

In [ ]:
service = build('gmail','v1',credentials=creds)
results = service.users().messages().list(userId='me',labelIds=['INBOX'],q="is:unread").execute()
messages = results.get('messages',[])
print(messages)

[]


In [38]:
def reply_message(sender,subject,text,message_id):
	try:
		service = build("gmail","v1",credentials=creds)
		message = EmailMessage()

		message.set_content(text)

		message["To"] = sender
		message["From"] = "roger.wang@biologybowl.org"
		message["Subject"] = subject
		message["In-Reply-To"] = message_id
		message["References"] = message_id

		encoded_message = base64.urlsafe_b64encode(message.as_bytes()).decode()

		create_message = {"raw": encoded_message}

		send_message = (
			service.users()
			.messages()
			.send(userId="me",body = create_message)
			.execute()
		)
		print(f'Message Id: {send_message["id"]}')

	except HttpError as error:
		print(f"An error occurred: {error}")
		send_message = None

In [39]:
if messages:
	for message in messages:
		msg = service.users().messages().get(userId='me',id = message['id']).execute()
		headers = msg['payload']['headers']

		from_email = None
		text = None
		subject = None
		message_id = None
		for header in headers:
			match header['name']:
				case 'From':
					from_name= header['value']
					from_email = re.findall('<(.*)>',from_name)[0]

					data = msg['payload']['parts'][0]['body']['data']	
					byte_code = base64.urlsafe_b64decode(data)
					text = str(byte_code.decode("utf-8"))
				
				case 'Subject':
					subject = header['value']

				case 'Message-ID':
					message_id = header['value']

		reply_message(from_email,subject,text,message_id)